<a href="https://colab.research.google.com/github/guscldns/Focus-Read/blob/main/Dount/json_to_text%2C_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 및 라이브러리 import

In [ ]:
# # 샘플파일
# !gdown 1rswWV67Idv8OXI1qQ7o5aSSWvP_uA5MW
# 맑음고딕 폰트
!gdown 1G4jAcJLy_1PC9YQR36t7jMapj9da4vPd

In [ ]:
# cuda 사용하도록 변경
import torch
print ( "CUDA 사용 가능:" , torch.cuda.is_available())
!nvcc --version

# 관련 라이브러리, 파일 설치
!git clone https://github.com/clovaai/donut.git
!cd donut && pip install

In [ ]:
!pip install gradio
!pip install timm==0.4.9
!pip install Transformers==4.25.1

In [ ]:
# 라이브러리
import zipfile
import os
import json
import torch
import argparse
import gradio as gr
from donut import DonutModel
from PIL import Image, ImageDraw, ImageFont
torch.cuda.is_available()

## 데이터 변환 : synthdog 사용

In [ ]:
# donut/synthdog 설치용
!pip install synthtiger

In [ ]:
import os
import json

foilderlist = os.listdir('./sample')

# 'kowiki.text' 파일 내용 초기화
with open(f'./donut/synthdog/resources/corpus/kowiki.txt', "w", encoding='utf-8') as fi:
    fi.truncate(0)
fi.close()

for i in foilderlist:
    per_path = f'./sample/{i}/20per'
    sent_path = f'./sample/{i}/2~3sent'
    per_list = os.listdir(per_path)
    sent_list = os.listdir(sent_path)

    # 원문만 불러오기
    for a, j in enumerate(per_list):
        with open(f'{per_path}/{j}', 'r', encoding='utf-8') as file:
            text = json.load(file)
            ground_truth_parse =  f" {text['Meta(Refine)']['passage']}"
            paasage = ground_truth_parse.replace('\n', ' ')

            # 원문 'kowiki.text' 에 덮어쓰기
            with open(f'./donut/synthdog/resources/corpus/kowiki.txt', 'a', encoding='utf-8') as f:
                f.write(f'{paasage} \n')

In [ ]:
%cd ./donut/synthdog
# 환경 변수 설정
!export OBJC_DISABLE_INITIALIZE_FORK_SAFETY=YES

# SynthDoG 실행
!synthtiger -o /home/alpaco/final_Project/donut/synthdog/outputs/KoreanData -c 100 -w 8 -v /home/alpaco/final_Project/donut/synthdog/template.py SynthDoG /home/alpaco/final_Project/donut/synthdog/config_ko.yaml

## dount

### 훈련

In [ ]:
# model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
## 모델 학습
# 만든 데이터 경로 train_cord.yaml에서 변경
# (예시) dataset_name_or_paths: [./donut/synthdog/outputs/KoreanData]
# cuda 사용하도록 변경
import torch
    # --resume_from_checkpoint_path "/home/alpaco/final_Project/donut/result/train_cord/20230904_113810/" \
!nvcc --version

!CUDA_VISIBLE_DEVICES=1 python /home/alpaco/final_Project/donut/train.py --config /home/alpaco/final_Project/donut/config/train_cord.yaml \
    --pretrained_model_name_or_path "naver-clova-ix/donut-base-finetuned-cord-v2" \
    --dataset_name_or_paths '["/home/alpaco/final_Project/donut/synthdog/outputs/KoreanData"]'\
    --resume_from_checkpoint_path "/home/alpaco/final_Project/donut/result/train_cord/test01/" \
    --result_path "/home/alpaco/final_Project/donut/result/" \
    --exp_version "test01"

### 테스트 파일로 성능 확인

In [ ]:
# test
!CUDA_VISIBLE_DEVICES=1 python /home/alpaco/final_Project/donut/test.py --dataset_name_or_path /home/alpaco/final_Project/donut/synthdog/outputs/KoreanData --pretrained_model_name_or_path /home/alpaco/final_Project/donut/result/train_cord/test01 --save_path /home/alpaco/final_Project/donut/result/output.json

## 실제 데이터로 테스트

In [ ]:
from donut import DonutModel
import torch
from PIL import Image


pretrained_model = DonutModel.from_pretrained("/home/alpaco/final_Project/donut/result/train_cord/test01")
if torch.cuda.is_available():
    pretrained_model.half()
    device = torch.device("cuda")
    pretrained_model.to(device)
else:
    pretrained_model.encoder.to(torch.bfloat16)
pretrained_model.eval()

task_name = "synthdog"
task_prompt = f"<s_{task_name}>"

input_img = Image.open("/home/alpaco/final_Project/donut/20230906_110217.jpg")
output = pretrained_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]
print(output)